In [117]:
!pip install --upgrade brasil-data
!pip install googletrans==4.0.0-rc1

from googletrans import Translator

# import python data science libs
# import pandas as pd
# import matplotlib.pyplot as plt

# imports sentiment Analyses
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# imports brazil market data
from brdata import xpi
from brdata import fundamentus

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Análise de sentimento

In [ ]:
class Platform:
  def __init__(self, url, tag, name):
    self.url = url
    self.tag = tag
    self.name = name

In [118]:
import re
# faz fetch das noticias e cria headlines
def fetch_headlines(platform: Platform, paper, console=False):
  response = requests.get(platform.url)
  soup = BeautifulSoup(response.content, "html.parser")
  headlines = soup.find_all(platform.tag, string=re.compile(paper, flags=re.IGNORECASE), limit=20)
  if console: print(f"Noticias lidas de {platform.name}: {len(headlines)}")

  return headlines

In [120]:
# traduz texto para inglês e melhora qualidade da análise do vader sentiment
def translate_text(text, target_language='en'):
    translator = Translator()
    translation = translator.translate(text, dest=target_language)
    return translation.text

In [122]:
class StockSentiment:
  def __init__(self, paper, console=False, text=False):
    self._paper = paper
    self._console = console
    self._text = text
    self._sentiment_score = 0
    self._total_score = 0
    self._action = "NEUTRO"

    self._platforms = [
        Platform(f"https://news.google.com/search?q={paper}",
                tag="a", name="Google News"),
        Platform(f"https://br.investing.com/search/?q={paper}&tab=news",
                tag="a", name="Investing.com"),
        Platform(f"https://einvestidor.estadao.com.br/?s={paper}",
                tag="h2", name="e-investidor"),
        Platform(f"https://exame.com/busca/{paper}/", tag="a", name="Exame"),
        Platform(f"https://www.seudinheiro.com/?s={paper}", tag="a", name="Seu Dinheiro"),
                ]

  def _calculate_sentiment(self, headlines):
    for headline in headlines:
      text = translate_text(headline.text)
      if self._text: print(text)
      analyzer = SentimentIntensityAnalyzer()
      sentiment = analyzer.polarity_scores(text)
      self._sentiment_score = self._sentiment_score + sentiment['compound']

    return self._sentiment_score

  def _analyse_platforms(self):
    for platform in self._platforms:
      headlines = fetch_headlines(platform, self._paper, self._console)
      self._sentiment_score = self._calculate_sentiment(headlines)
      self._total_score = self._total_score + self._sentiment_score
      if self._console: print(f"Sentimento {platform.name}: {self._sentiment_score} \n")


  def stock_analysis(self):
    self._analyse_platforms()
    media = self._total_score / len(self._platforms)

    if self._console:
      print(f"Total: {round(self._total_score, 2)}")
      print(f"Média Sobre Total: {round(media, 2)}\n")

    analyse = xpi.analise(self._paper)
    if media > 0.8: self._action = "COMPRA"
    elif media < -0.8: self._action = "VENDA"

    return {
        "action":
            self._action,
        "xpi_analise":
            analyse,
        "fundamentus":
            fundamentus.detalhes(self._paper.upper())
        }

In [123]:
stock = StockSentiment("bbdc4").stock_analysis()
print(f"Business: { stock['fundamentus']['Empresa'] }")
print(f"Setor: { stock['fundamentus']['Setor'] }")
print(f"Action: { stock['action'] }")
print(f"Action XPI: { stock['xpi_analise']['Recomendação'] }")

Business: BANCO BRADESCO S.A. PN N1
Setor: Intermediários Financeiros
Action: NEUTRO
Action XPI: neutro


# Análise de tendência


In [16]:

# # Importe os dados da ação
# dados = pd.read_csv("dados_acao.csv")

# # Calcule a média móvel de 50 períodos
# media_movel_50 = dados["preco"].rolling(50).mean()

# # Calcule a média móvel de 200 períodos
# media_movel_200 = dados["preco"].rolling(200).mean()

# # Plote os dados
# plt.plot(dados["data"], dados["preco"], label="Preço")
# plt.plot(dados["data"], media_movel_50, label="Média móvel de 50 períodos")
# plt.plot(dados["data"], media_movel_200, label="Média móvel de 200 períodos")
# plt.legend()
# plt.show()

# # Analise a tendência
# if media_movel_50 > media_movel_200:
#   print("A tendência é de alta.")
# elif media_movel_50 < media_movel_200:
#   print("A tendência é de baixa.")
# else:
#   print("A tendência é lateral.")


# Calculo de risco

In [17]:
# def calcula_risco(dados):
#   """
#   Calcula o risco de uma ação usando o desvio padrão.

#   Args:
#     dados: Uma série de dados com os retornos da ação.

#   Returns:
#     O desvio padrão dos retornos da ação.
#   """

#   retornos = dados["Retorno"]
#   desvio_padrao = retornos.std()
#   return desvio_padrao

# # Exemplo de uso

# dados = pd.read_csv("dados.csv")
# risco = calcula_risco(dados)
# print(risco)